In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Incidents

loop

In [ ]:
raw_path = '../data/PeMS/Incidents/raw'
dest_path = '../data/PeMS/Incidents/light'
pth = Path(raw_path)
for child in pth.iterdir():
    # incidents data reading
    incdnt_file_name = child.name
    inc_header = ['IncidentID', 'CC_Code', 'Incident_No', 'Timestamp', 'Description', 'Location', 'Area', 'Zoom_Map', 'TBxy', 'Latitude', 'Longitude', 'District', 'CountryFIPS_ID', 'CityFIPS_ID', 'Freeway', 'Freeway_direction', 'State_postmile', 'Absolute_postmile', 'Severity', 'Duration', 'Incident_ID', 'Detail_ID', 'Timestamp', 'description']
    data_inc_d07 = pd.read_csv(raw_path+'/'+incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
    data_inc_d07 = (data_inc_d07.dropna(subset=['District']))[data_inc_d07.columns[:-4]]
    data_inc_d07 = data_inc_d07.astype(dtype={'District':int})
    # result: incidents in district under analysis
    data_inc_d07 = data_inc_d07[data_inc_d07['District']==7].reset_index(drop=True)
    data_inc_d07.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)

single

In [2]:
# incidents data reading
incdnt_file_name = '../data/PeMS/Incidents/raw/all_text_chp_incident_day_2017_03_06.txt.gz'
inc_header = ['IncidentID', 'CC_Code', 'Incident_No', 'Timestamp', 'Description', 'Location', 'Area', 'Zoom_Map', 'TBxy', 'Latitude', 'Longitude', 'District', 'CountryFIPS_ID', 'CityFIPS_ID', 'Freeway', 'Freeway_direction', 'State_postmile', 'Absolute_postmile', 'Severity', 'Duration', 'Incident_ID', 'Detail_ID', 'Timestamp', 'description']
data_inc_d07 = pd.read_csv(incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
data_inc_d07 = (data_inc_d07.dropna(subset=['District']))[data_inc_d07.columns[:-4]]
data_inc_d07 = data_inc_d07.astype(dtype={'District':int})
# result: incidents in district under analysis
data_inc_d07 = data_inc_d07[data_inc_d07['District']==7].reset_index(drop=True)

In [3]:
data_inc_d07.to_csv('../data/PeMS/Incidents/light/all_text_chp_incident_day_2017_03_06.csv', index=False)

# Incidents details

In [48]:
incdnt_det_file_name = '../data/PeMS/Incidents/all_text_chp_incident_det_day_2017_10_11.txt'
det_header = ['ID', 'DetID', 'Timestamp', 'Desc']
data_inc_det_d07 = pd.read_csv(incdnt_det_file_name, sep=',', names=det_header, parse_dates=[2])

In [50]:
data_inc_det_d07.to_csv('../data/PeMS/Incidents/light/all_text_chp_incident_det_day_2017_10_11.csv', index=False)

# Series

loop

In [24]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
raw_path = '../data/PeMS/Series/raw'
raw_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/2017/May/'
dest_path = '../data/PeMS/Series/light'
dest_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/light/May'
pth = Path(raw_path)
st_blacklist = np.array([])
header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
          'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
          'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
          'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
          'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
          'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
          'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
          'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
          'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
          'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']
for child in pth.iterdir():
    # incidents data reading
    srs_file_name = child.name
#     if srs_file_name[0] !='p':
#         continue
    print(srs_file_name)

    data_srs_smoothed = pd.read_csv(raw_path+'/'+srs_file_name, sep=',', names=header_srs, parse_dates=[0])

    for col in data_srs_smoothed.columns[-40:]:
        del data_srs_smoothed[col]
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)
    data_srs_smoothed.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)
    #smoothing
#     data_srs_smoothed = data_srs_smoothed[data_srs_smoothed.columns[:16]]
#     del data_srs_no_null

    gb = data_srs_smoothed.groupby(by=['Station'])
    j=0
    for i, item in gb:
    #     print(item['Avg Speed'].rolling(5).mean())
    #     station = item.Station[0]
        data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
    #     j += 1
    #     if j==2:
    #         break
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)

#     del srs_for_smoothing_gb

    gb = data_srs_smoothed.groupby(by=['Station'])
    non_sensitive_sts = gb.filter(lambda x: (x['Avg Speed'].std()<1))['Station'].unique()
    data_srs_smoothed = data_srs_smoothed[~data_srs_smoothed.Station.isin(non_sensitive_sts)]
    data_srs_smoothed.to_csv(dest_path+'/smoothed/'+child.name[:-4]+'.csv', index=False)

    data_srs_smoothed = None

    st_blacklist = np.append(st_blacklist, non_sensitive_sts[~np.isin(non_sensitive_sts, st_blacklist)])
    gb, non_sensitive_sts = None, None

d07_text_station_5min_2017_05_28.txt.gz
d07_text_station_5min_2017_05_24.txt.gz
d07_text_station_5min_2017_05_30.txt.gz
d07_text_station_5min_2017_05_27.txt.gz
d07_text_station_5min_2017_05_12.txt.gz
d07_text_station_5min_2017_05_23.txt.gz
d07_text_station_5min_2017_05_31.txt.gz
d07_text_station_5min_2017_05_14.txt.gz
d07_text_station_5min_2017_05_29.txt.gz
d07_text_station_5min_2017_05_04.txt.gz
d07_text_station_5min_2017_05_06.txt.gz
d07_text_station_5min_2017_05_16.txt.gz
d07_text_station_5min_2017_05_19.txt.gz
d07_text_station_5min_2017_05_01.txt.gz
d07_text_station_5min_2017_05_07.txt.gz
d07_text_station_5min_2017_05_22.txt.gz
d07_text_station_5min_2017_05_21.txt.gz
d07_text_station_5min_2017_05_03.txt.gz
d07_text_station_5min_2017_05_18.txt.gz
d07_text_station_5min_2017_05_08.txt.gz
d07_text_station_5min_2017_05_26.txt.gz
d07_text_station_5min_2017_05_13.txt.gz
d07_text_station_5min_2017_05_11.txt.gz
d07_text_station_5min_2017_05_02.txt.gz
d07_text_station_5min_2017_05_17.txt.gz


In [3]:
np.savetxt(dest_path+'/stations_blacklist_may.csv', st_blacklist)

In [4]:
raw_path = '../data/PeMS/Series/raw'
raw_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/2017/Jun/'
dest_path = '../data/PeMS/Series/light'
dest_path = '/home/frolovconst/Dvlpt/prj/Python/tc_w_ml/data/PeMS/Series/Batch/light/Jun'
pth = Path(raw_path)
st_blacklist = np.array([])
header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
          'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
          'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
          'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
          'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
          'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
          'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
          'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
          'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
          'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']
for child in pth.iterdir():
    # incidents data reading
    srs_file_name = child.name
#     if srs_file_name[0] !='p':
#         continue
    print(srs_file_name)

    data_srs_smoothed = pd.read_csv(raw_path+'/'+srs_file_name, sep=',', names=header_srs, parse_dates=[0])

    for col in data_srs_smoothed.columns[-40:]:
        del data_srs_smoothed[col]
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)
    data_srs_smoothed.to_csv(dest_path+'/'+child.name[:-4]+'.csv', index=False)
    #smoothing
#     data_srs_smoothed = data_srs_smoothed[data_srs_smoothed.columns[:16]]
#     del data_srs_no_null

    gb = data_srs_smoothed.groupby(by=['Station'])
    j=0
    for i, item in gb:
    #     print(item['Avg Speed'].rolling(5).mean())
    #     station = item.Station[0]
        data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
    #     j += 1
    #     if j==2:
    #         break
    data_srs_smoothed.dropna(subset=['Avg Speed'], inplace=True)

#     del srs_for_smoothing_gb

    gb = data_srs_smoothed.groupby(by=['Station'])
    non_sensitive_sts = gb.filter(lambda x: (x['Avg Speed'].std()<1))['Station'].unique()
    data_srs_smoothed = data_srs_smoothed[~data_srs_smoothed.Station.isin(non_sensitive_sts)]
    data_srs_smoothed.to_csv(dest_path+'/smoothed/'+child.name[:-4]+'.csv', index=False)

    data_srs_smoothed = None

    st_blacklist = np.append(st_blacklist, non_sensitive_sts[~np.isin(non_sensitive_sts, st_blacklist)])
    gb, non_sensitive_sts = None, None

d07_text_station_5min_2017_06_19.txt.gz
d07_text_station_5min_2017_06_05.txt.gz
d07_text_station_5min_2017_06_25.txt.gz
d07_text_station_5min_2017_06_11.txt.gz
d07_text_station_5min_2017_06_21.txt.gz
d07_text_station_5min_2017_06_28.txt.gz
d07_text_station_5min_2017_06_12.txt.gz
d07_text_station_5min_2017_06_16.txt.gz
d07_text_station_5min_2017_06_17.txt.gz
d07_text_station_5min_2017_06_18.txt.gz
d07_text_station_5min_2017_06_29.txt.gz
d07_text_station_5min_2017_06_01.txt.gz
d07_text_station_5min_2017_06_02.txt.gz
d07_text_station_5min_2017_06_04.txt.gz
d07_text_station_5min_2017_06_15.txt.gz
d07_text_station_5min_2017_06_13.txt.gz
d07_text_station_5min_2017_06_24.txt.gz
d07_text_station_5min_2017_06_09.txt.gz
d07_text_station_5min_2017_06_08.txt.gz
d07_text_station_5min_2017_06_14.txt.gz
d07_text_station_5min_2017_06_26.txt.gz


MemoryError: 

In [ ]:
np.savetxt(dest_path+'/stations_blacklist_jun.csv', st_blacklist)

single

In [32]:
# traffic flow data reading
header_srs = ['Timestamp', 'Station','District', 'Freeway', 'Direction of Travel', 'Lane Type', 'Station Length',
          'Samples', '% Observed', 'Total Flow', 'Avg Occupancy', 'Avg Speed', 
          'Lane 1 Samples', 'Lane 1 Flow', 'Lane 1 Avg Occ', 'Lane 1 Avg Speed', 'Lane 1 Observed',
          'Lane 2 Samples', 'Lane 2 Flow', 'Lane 2 Avg Occ', 'Lane 2 Avg Speed', 'Lane 2 Observed',
          'Lane 3 Samples', 'Lane 3 Flow', 'Lane 3 Avg Occ', 'Lane 3 Avg Speed', 'Lane 3 Observed',
          'Lane 4 Samples', 'Lane 4 Flow', 'Lane 4 Avg Occ', 'Lane 4 Avg Speed', 'Lane 4 Observed',
          'Lane 5 Samples', 'Lane 5 Flow', 'Lane 5 Avg Occ', 'Lane 5 Avg Speed', 'Lane 5 Observed',
          'Lane 6 Samples', 'Lane 6 Flow', 'Lane 6 Avg Occ', 'Lane 6 Avg Speed', 'Lane 6 Observed',
          'Lane 7 Samples', 'Lane 7 Flow', 'Lane 7 Avg Occ', 'Lane 7 Avg Speed', 'Lane 7 Observed',
          'Lane 8 Samples', 'Lane 8 Flow', 'Lane 8 Avg Occ', 'Lane 8 Avg Speed', 'Lane 8 Observed']

srs_file_name = '../data/PeMS/Series/d07_text_station_5min_2017_10_11.txt'
data_srs_no_null = pd.read_csv(srs_file_name, sep=',', names=header_srs, parse_dates=[0])
data_srs_no_null = data_srs_no_null[data_srs_no_null.columns[:-40]]
data_srs_no_null = data_srs_no_null.dropna(subset=['Avg Speed']).reset_index(drop=True)

In [33]:
data_srs_no_null.to_csv('../data/PeMS/Series/light/d07_text_station_5min_2017_10_11.csv', index=False)

In [34]:
data_srs_smoothed = data_srs_no_null[data_srs_no_null.columns[:16]].copy()
srs_for_smoothing_gb = data_srs_smoothed.groupby(by=['Station'])
j=0
for i, item in srs_for_smoothing_gb:
#     print(item['Avg Speed'].rolling(5).mean())
#     station = item.Station[0]
    data_srs_smoothed.loc[item.index, ('Avg Speed')] = item['Avg Speed'].rolling(5).mean()
#     j += 1
#     if j==2:
#         break
data_srs_smoothed = data_srs_smoothed.dropna(subset=['Avg Speed'])
del srs_for_smoothing_gb

In [35]:
data_srs_smoothed.to_csv('../data/PeMS/Series/smoothed/d07_text_station_5min_2017_10_11.csv', index=False)

# Stations

In [5]:
file_stations = '../data/PeMS/Stations/d07_text_meta_2017_09_20.txt'
header_stations = ['ID', 'Freeway', 'Freeway dir', 'Country', 'City', 'State Postmile', 'Abs Postmile', 
          'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name',
          'User ID1', 'User ID2', 'User ID3', 'User ID4']
stations_active = pd.read_csv(file_stations, sep='\t')
stations_active = stations_active.astype({'Latitude':float, 'Longitude':float})
stations_active = stations_active.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
# result: non-empty speed data series  only, stations which provide these data
stations_active = stations_active[stations_active['ID'].isin(data_srs_no_null['Station'].unique())].reset_index(drop=True)
stations_active = stations_active.sort_values('Lanes', ascending=False).drop_duplicates(subset=['Latitude', 'Longitude'], keep='first').sort_index().reset_index(drop=True)
stations_active = stations_active[~stations_active.ID.isin(st_blacklist)]

In [6]:
stations_active.to_csv('../data/PeMS/Stations/light/d07_text_meta_2017_09_20_active.csv', index=False)

# Do not use

In [38]:
# incidents week before data reading
incdnt_file_name = '../data/PeMS/Incidents/all_text_chp_incident_day_2017_10_04.txt'
data_inc_04_d07 = pd.read_csv(incdnt_file_name, sep=',', names=inc_header, parse_dates=[3])
data_inc_04_d07 = (data_inc_04_d07.dropna(subset=['District']))[data_inc_04_d07.columns[:-4]]
data_inc_04_d07 = data_inc_04_d07.astype(dtype={'District':int})
# result: incidents in district under analysis
data_inc_04_d07 = data_inc_04_d07[data_inc_04_d07['District']==7].reset_index(drop=True)

In [39]:
data_inc_04_d07.to_csv('../data/PeMS/Incidents/light/all_text_chp_incident_day_2017_10_04.csv', index=False)

In [40]:
# traffic week b4 flow data reading
srs_file_name = '../data/PeMS/Series/d07_text_station_5min_2017_10_04.txt'
data_srs_no_null_04 = pd.read_csv(srs_file_name, sep=',', names=header_srs, parse_dates=[0])
data_srs_no_null_04 = data_srs_no_null_04[data_srs_no_null_04.columns[:-40]]
# result: non-empty speed data series  only, stations which provide these data
data_srs_no_null_04 = data_srs_no_null_04.dropna(subset=['Avg Speed']).reset_index(drop=True)

In [41]:
data_srs_no_null_04.to_csv('../data/PeMS/Series/light/d07_text_station_5min_2017_10_04.csv', index=False)